In [38]:
import pandas as pd

In [ ]:
import pandas as pd

files = ['valeursFoncieres-2019-S2.txt', 'ValeursFoncieres-2020.txt']
chunks = []

for file in files:
    chunk_iter = pd.read_csv(file, sep="|", low_memory=True, chunksize=100000)  
    for chunk in chunk_iter:
        chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)
pd.set_option('display.max_columns', None)  



C:\Users\HP\AppData\Local\Temp\ipykernel_20792\2232227314.py:8: DtypeWarning: Columns (24,26,28,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunk_iter:
C:\Users\HP\AppData\Local\Temp\ipykernel_20792\2232227314.py:8: DtypeWarning: Columns (23,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunk_iter:
C:\Users\HP\AppData\Local\Temp\ipykernel_20792\2232227314.py:8: DtypeWarning: Columns (24,26,29,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunk_iter:
C:\Users\HP\AppData\Local\Temp\ipykernel_20792\2232227314.py:8: DtypeWarning: Columns (31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunk_iter:
C:\Users\HP\AppData\Local\Temp\ipykernel_20792\2232227314.py:8: DtypeWarning: Columns (23,29,33) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in chunk_iter:
C

   Identifiant de document  Reference document  1 Articles CGI  \
0                      NaN                 NaN             NaN   
1                      NaN                 NaN             NaN   
2                      NaN                 NaN             NaN   
3                      NaN                 NaN             NaN   
4                      NaN                 NaN             NaN   

   2 Articles CGI  3 Articles CGI  4 Articles CGI  5 Articles CGI  \
0             NaN             NaN             NaN             NaN   
1             NaN             NaN             NaN             NaN   
2             NaN             NaN             NaN             NaN   
3             NaN             NaN             NaN             NaN   
4             NaN             NaN             NaN             NaN   

   No disposition Date mutation Nature mutation Valeur fonciere  No voie  \
0               1    01/07/2019           Vente       244200,00     73.0   
1               1    04/07/2019     

In [ ]:
df.head()
#nombre de valeur manquante
df.isnull().sum() * 100 / len(df)

print(type(df["Valeur fonciere"][0]))

df["Valeur fonciere"] = df["Valeur fonciere"].str.replace(",", ".").astype(float)


AttributeError: Can only use .str accessor with string values!

In [21]:
#observer les valeurs abeérantes
print(df['Valeur fonciere'].describe())


#nombre de valeurs abérrantes
Q1 = df['Valeur fonciere'].quantile(0.25)
Q3 = df['Valeur fonciere'].quantile(0.75)
IQR = Q3 - Q1
print('nombre de valeurs abérrantes : ', df[(df['Valeur fonciere'] < (Q1 - 1.5 * IQR)) | (df['Valeur fonciere'] > (Q3 + 1.5 * IQR))].shape[0])

#supprimer les valeurs abérrantes
def supprimer_valeur_abrerantes(df , nom_colonne):
    Q1 = df[nom_colonne].quantile(0.25)
    Q3 = df[nom_colonne].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df[nom_colonne] > (Q1 - 1.5 * IQR)) & (df[nom_colonne] < (Q3 + 1.5 * IQR))]
    return df
   
df = supprimer_valeur_abrerantes(df, 'Valeur fonciere')   

print("nombre de valeurs abérrantes après suppression : ", df[(df['Valeur fonciere'] < (Q1 - 1.5 * IQR)) | (df['Valeur fonciere'] > (Q3 + 1.5 * IQR))].shape[0])

count    1.945453e+06
mean     1.006466e+06
std      8.090271e+06
min      1.000000e-02
25%      6.450000e+04
50%      1.550000e+05
75%      2.821490e+05
max      8.450000e+08
Name: Valeur fonciere, dtype: float64
nombre de valeurs abérrantes :  201232
nombre de valeurs abérrantes après suppression :  0


In [22]:
valeur_null = df.isnull().sum() * 100 / len(df)
print(valeur_null)

Identifiant de document       100.000000
Reference document            100.000000
1 Articles CGI                100.000000
2 Articles CGI                100.000000
3 Articles CGI                100.000000
4 Articles CGI                100.000000
5 Articles CGI                100.000000
No disposition                  0.000000
Date mutation                   0.000000
Nature mutation                 0.000000
Valeur fonciere                 0.000000
No voie                        41.467108
B/T/Q                          95.662820
Type de voie                   43.022530
Code voie                       0.842038
Voie                            0.842611
Code postal                     0.847828
Commune                         0.000000
Code departement                0.000000
Code commune                    0.000000
Prefixe de section             95.273764
Section                         0.003612
No plan                         0.000000
No Volume                      99.807479
1er lot         

Nettoyer les données

In [23]:
#supprimer les colonnes qui plus de 90% des valeurs sont manquantes
for i in valeur_null.index:
    if valeur_null[i] > 90:
        df = df.drop(i, axis=1)

In [24]:
#supprimer les doublons
df = df.drop_duplicates()
df.shape


(1706606, 22)

In [25]:
df.drop('1er lot', axis=1, inplace=True)


In [26]:
df.isnull().sum() * 100 / len(df)
#summrier les lignes qui ont des valeurs manquantes
df.dropna(subset=['Code postal', 'Commune', 'Surface reelle bati', 'Nombre pieces principales', 'Type local'], inplace=True)
df.isnull().sum() * 100 / len(df)

df.dropna(subset=['No voie','Type de voie','Nature culture' , 'Surface terrain'], inplace=True)

In [27]:
df.isnull().sum() * 100 / len(df)

No disposition               0.000000
Date mutation                0.000000
Nature mutation              0.000000
Valeur fonciere              0.000000
No voie                      0.000000
Type de voie                 0.000000
Code voie                    0.000000
Voie                         0.000000
Code postal                  0.000000
Commune                      0.000000
Code departement             0.000000
Code commune                 0.000000
Section                      0.002758
No plan                      0.000000
Nombre de lots               0.000000
Code type local              0.000000
Type local                   0.000000
Surface reelle bati          0.000000
Nombre pieces principales    0.000000
Nature culture               0.000000
Surface terrain              0.000000
dtype: float64

In [28]:
#convertir les colonnes en type numerique
df['Valeur fonciere'] = pd.to_numeric(df['Valeur fonciere'], errors='coerce')
df['Surface reelle bati'] = pd.to_numeric(df['Surface reelle bati'], errors='coerce')
df['Nombre pieces principales'] = pd.to_numeric(df['Nombre pieces principales'], errors='coerce')
df['Surface terrain'] = pd.to_numeric(df['Surface terrain'], errors='coerce')


df['Date mutation'] = pd.to_datetime(df['Date mutation'], errors='coerce')
print(type(df['Date mutation'][0]))


df.head(10)

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,...,Code commune,Section,No plan,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain
0,1,2019-01-07,Vente,244200.0,73.0,RUE,0958,COMMANDANT COUSTEAU,1000.0,BOURG-EN-BRESSE,...,53,CN,405,0,1.0,Maison,150.0,5.0,S,695.0
1,1,2019-04-07,Vente,195000.0,852.0,CHE,0170,DE LA CROIX ROSIER,1250.0,HAUTECOURT-ROMANECHE,...,184,AM,237,0,1.0,Maison,94.0,4.0,AG,888.0
2,1,2019-04-07,Vente,195000.0,852.0,CHE,0170,DE LA CROIX ROSIER,1250.0,HAUTECOURT-ROMANECHE,...,184,AM,237,0,1.0,Maison,94.0,4.0,S,500.0
8,1,2019-01-07,Vente,211500.0,1259.0,CHE,0031,DE LA CAMBUSE,1440.0,VIRIAT,...,451,E,1673,0,1.0,Maison,108.0,4.0,S,1159.0
9,1,2019-02-07,Vente,180000.0,97.0,RUE,0605,BRICHEMER,1000.0,BOURG-EN-BRESSE,...,53,BX,267,0,1.0,Maison,97.0,5.0,S,314.0
10,1,2019-02-07,Vente,216600.0,1.0,AV,3160,MARECHAL JUIN,1000.0,BOURG-EN-BRESSE,...,53,CR,68,0,3.0,Dépendance,0.0,0.0,S,1206.0
11,1,2019-02-07,Vente,216600.0,1.0,AV,3160,MARECHAL JUIN,1000.0,BOURG-EN-BRESSE,...,53,CR,68,0,2.0,Appartement,25.0,1.0,S,1206.0
12,1,2019-02-07,Vente,216600.0,1.0,AV,3160,MARECHAL JUIN,1000.0,BOURG-EN-BRESSE,...,53,CR,68,0,2.0,Appartement,73.0,4.0,S,1206.0
13,1,2019-01-07,Vente,206473.0,54.0,ALL,2617,DES MARTINETS,1000.0,BOURG-EN-BRESSE,...,53,CW,305,0,1.0,Maison,91.0,4.0,S,728.0
14,1,2019-01-07,Vente,195690.0,616.0,RTE,0151,DE MARBOZ,1440.0,VIRIAT,...,451,AX,153,0,1.0,Maison,105.0,5.0,S,952.0


In [29]:
#supprimer les lignes ou la valeurs fonciere est 0 ou 1
df = df.loc[df['Valeur fonciere'] > 1] 

df.shape



(361569, 21)